In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,auc  #常见的分类评分标准
from sklearn.model_selection import train_test_split #切分数据
from sklearn.linear_model import LogisticRegression #逻辑回归
from sklearn.model_selection import GridSearchCV as gscv #网格搜索
from sklearn.neighbors import KNeighborsClassifier #k近邻
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
import glob
import math
import seaborn as sns 
import matplotlib.pyplot as plt
import xgboost as xgb 
import toad
import pickle


ModuleNotFoundError: No module named 'xgboost'